# Neural Network binary classifier
We build a Multilayer Perceptron binary classifier using Keras to predict Survival in the Titanic Dataset. Even though this might not be the optimal classifier for this dataset, this notebook's purpose is to get acquainted with the Keras API to build classifiers. In comparison with other methods it performs quite well though, albeit requiring a bit of tinkering with the hyperparameters.

In [1]:
import pandas as pd
import numpy as np
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils  # for transforming data later
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

seed = 42  # for reproducability
np.random.seed(seed)  

Using TensorFlow backend.


load data and show first 10 rows of the training data set

In [2]:
train = pd.read_csv( '../input/train.csv')
test = pd.read_csv('../input/test.csv')

n_train = train.shape[0]
n_test = test.shape[0]


## Feature engineering
The original data set has gaps, and information that in its original form is not very useful. We impute missing values and transform to more useful features (using ideas from Sina's Kernel)

In [3]:
full_data = [train, test]

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# some of the feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column (use most common value 'S')
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and fill with median
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
# Remove all NULLS in the Age column and fill random age in range[avg - std, avg + std] (questionable...)
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()  # counts the number of NaN is the Age column
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # fill missing ages with random ages around mean age
    dataset.loc[dataset['Age'].isnull(), 'Age'] = age_null_random_list  
    dataset['Age'] = dataset['Age'].astype(int)
# Map Sex to categorical [1,0]
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].astype("category").cat.codes
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].astype("category").cat.codes

# create dummies for Embarked (unordered categorical feature)
# why does this not work if done in the above for loop (it has no effect)
# train = pd.get_dummies(train, columns=['Embarked'])
# test = pd.get_dummies(test, columns=['Embarked'])
    

## Cleaned Dataset
we drop unnecessary features to obtain a cleaned dataset containing only numerical values

In [4]:
drop_elements = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Has_Cabin,FamilySize,IsAlone
0,1,0,3,1,22,7.2500,2,0,2,0
1,2,1,1,0,38,71.2833,0,1,2,0
2,3,1,3,0,26,7.9250,2,0,1,1
3,4,1,1,0,35,53.1000,2,1,2,0
4,5,0,3,1,35,8.0500,2,0,1,1
5,6,0,3,1,21,8.4583,1,0,1,1
6,7,0,1,1,54,51.8625,2,1,1,1
7,8,0,3,1,2,21.0750,2,0,5,0
8,9,1,3,0,27,11.1333,2,0,3,0
9,10,1,2,0,14,30.0708,0,0,2,0


## Select wanted features
Here we choose the features of the full cleaned dataset that we actually want to use for fitting

In [6]:
#wanted_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Has_Cabin', 'FamilySize', 'IsAlone',\
                   #'Embarked_C', 'Embarked_Q', 'Embarked_S']
wanted_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Has_Cabin', 'IsAlone', 'Embarked']
#wanted_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Has_Cabin', 'IsAlone', 'Embarked_C', 'Embarked_Q', 'Embarked_S']

X_train = train[wanted_features].values
X_test = test[wanted_features].values

Y_train = train['Survived'].values

n_features = X_train.shape[1]

## Keras MLP binary classifier

define the network architecure

In [7]:
def build_model(hidden_dims=[10], dropouts=[0], optim='adam', verbose=True):
    model = Sequential()
    model.add(Dense(hidden_dims[0], input_dim=n_features, activation='relu', kernel_initializer='normal'))
    if dropouts[0] > 0.:
        model.add(Dropout(dropouts[0]))
    
    for i in range(1,len(hidden_dims)):
            model.add(Dense(hidden_dims[i], activation='relu', kernel_initializer='normal'))
            if dropouts[i] > 0.:
                model.add(Dropout(dropouts[i]))
    
    model.add(Dense(1, activation='sigmoid', kernel_initializer='normal'))  
    if verbose:
        print(model.summary())  
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

fit the model

In [8]:
epochs = 100
batch_size = 5

sk_params = {'hidden_dims': [n_features, int(n_features/2)], 'dropouts': [0, 0]}
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', 
                   KerasClassifier(build_fn=build_model, epochs=epochs, batch_size=batch_size, verbose=1, **sk_params)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
print("\n\n Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/valentin/anaconda3/envs/TF/lib/python3.6/site-packages/sklearn/base.py:114: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/home/valentin/anaconda3/envs/TF/lib/python3.6/site-packages/sklearn/base.py:114: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 24        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4         
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________
None


Epoch 1/100


  5/801 [..............................] - ETA: 49s - loss: 0.6931 - acc: 0.4000

190/801 [======>.......................] - ETA: 1s - loss: 0.6909 - acc: 0.6526 

...

205/803 [======>.......................] - ETA: 0s - loss: 0.3808 - acc: 0.8244

395/803 [=============>................] - ETA: 0s - loss: 0.4037 - acc: 0.8177

590/803 [=====================>........] - ETA: 0s - loss: 0.4370 - acc: 0.8068

780/803 [============================>.] - ETA: 0s - loss: 0.4343 - acc: 0.8167

803/803 [==============================] - 0s - loss: 0.4352 - acc: 0.8169     

In [1]:

print("\n\n Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

NameError: name 'results' is not defined

In [14]:
Y_pred = model.predict(X_test)
surv_msk = Y_pred > 0.5
Y_pred = np.zeros(Y_pred.shape)
Y_pred[surv_msk] = 1.

n_train = len(Y_train)
n_test = len(Y_test)
train_frac = Y_train.sum()/n_train
pred_frac = Y_pred.sum()/n_train
test_frac = Y_test.sum()/n_train

print('test set contains %i samples' % n_test)

print('%2.2f%% survived in training set' % (100*train_frac))
print('%2.2f%% survived in test set' % (100*test_frac))
print('%2.2f%% survived in predicted test set' % (100*pred_frac))

acc = accuracy_score(Y_test, Y_pred)
print('Test Accuracy: %2.2f%%' % (100*acc))

test set contains 418 samples
36.54% survived in training set
41.76% survived in test set
37.09% survived in predicted test set
Test Accuracy: 88.76%


In [15]:
cm = confusion_matrix(Y_test, Y_pred)
f_score = f1_score(Y_test, Y_pred)
print('Confusion Matrix:')
print(cm)

cm_percent = 100*cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
print('\nin percent per row:')
print(cm_percent)

print('\n')
print('%02.2f%% of deceased predicted correctly (true negative)' % cm_percent[0,0])
print('%02.2f%% of deceased predicted INcorrectly (false negative)' % cm_percent[0,1])

print('\n')
print('%02.2f%% of survived predicted correctly (true positive)' % cm_percent[1,1])
print('%02.2f%% of survived predicted INcorrectly (false positive)' % cm_percent[1,0])

print('\nF-Score: %2.4f' % (f_score))

Confusion Matrix:
[[251  15]
 [ 32 120]]

in percent per row:
[[ 94.36090226   5.63909774]
 [ 21.05263158  78.94736842]]


94.36% of deceased predicted correctly (true negative)
5.64% of deceased predicted INcorrectly (false negative)


78.95% of survived predicted correctly (true positive)
21.05% of survived predicted INcorrectly (false positive)

F-Score: 0.8362


In [16]:
Y_final = model.predict(X_test)
surv_msk = Y_final > 0.5
Y_final = np.zeros(Y_final.shape)
Y_final[surv_msk] = 1
Y_final.shape

test['Survived'] = Y_final.astype('int64')
df_out = test[['PassengerId', 'Survived']]
df_out.head(10)
df_out.to_csv('../output/predictions_MLP_Keras.csv', index=False)